In [ ]:
# %matplotlib inline
# %matplotlib qt5
%matplotlib widget
import numpy as np
from eval_and_vis import eval_vis_model
import matplotlib.pyplot as plt
import asyncio

In [ ]:
config_path = './work_dir/ntu60/dstanet_drop0_6090120_128/ntu60_dstanet.yaml'
vid = 'S004C001P003R001A032'

In [ ]:
data, edge, is_3d, tag = eval_vis_model(vid = vid, config_path=config_path)

In [ ]:
N, C, T, V, M = data.shape
plt.ion()
fig = plt.figure()
fig.suptitle(tag)
if is_3d:
    from mpl_toolkits.mplot3d import Axes3D
    ax = fig.add_subplot(111, projection='3d')
else:
    ax = fig.add_subplot(111)
p_type = ['b-', 'g-', 'r-', 'c-', 'm-', 'y-', 'k-', 'k-', 'k-', 'k-']
pose = []
for m in range(M):
    a = []
    for i in range(len(edge)):
        if is_3d:
            a.append(ax.plot(np.zeros(3), np.zeros(3), p_type[m])[0])
        else:
            a.append(ax.plot(np.zeros(2), np.zeros(2), p_type[m])[0])
    pose.append(a)
ax.axis([-1, 1, -1, 1])
if is_3d:
    ax.set_zlim3d(-1, 1)
plt.axis('on')
async def update():
    for t in range(T):
        for m in range(M):
            for i, (v1, v2) in enumerate(edge):
                x1 = data[0, :2, t, v1, m]
                x2 = data[0, :2, t, v2, m]
                if (x1.sum() != 0 and x2.sum() != 0) or v1 == 1 or v2 == 1:
                    pose[m][i].set_xdata(data[0, 0, t, [v1, v2], m])
                    pose[m][i].set_ydata(data[0, 1, t, [v1, v2], m])
                    if is_3d:
                        pose[m][i].set_3d_properties(data[0, 2, t, [v1, v2], m])
        fig.canvas.draw()
        await asyncio.sleep(0.02)
loop = asyncio.get_event_loop()
loop.create_task(update())
plt.close()
plt.ioff()
print('end')